## Initialize (Variables and Libraries)

In [18]:
PROJECT_ID = "vtxdemos"
STAGING_FOLDER_URI =  "gs://vtxdemos-staging"
IMAGE_URI = "gcr.io/vtxdemos/tensorflow-gpu-nlp:v1"
MODEL_URI = "gs://vtxdemos-models/nlp"

In [2]:
from google.cloud import aiplatform as aip

## Creating Folder Structure for Training Script

In [3]:
!rm -fr training
!mkdir training

In [4]:
%%writefile training/train.py
#%%
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from google.cloud import bigquery
import tensorflow_datasets as tfds
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

client = bigquery.Client(project="vtxdemos")

## Loading testing dataset from bigquery
sql = "select * from `public.train_nlp`"
train_df = client.query(sql).to_dataframe()
train_examples = np.array([i.encode('utf-8') for i in train_df['text']], dtype="object")
train_labels = train_df['labels'].to_numpy(dtype=int)

## Loading testing dataset from bigquery
sql = "select * from `vtxdemos.public.train_nlp`"
test_df = client.query(sql).to_dataframe()
test_examples = np.array([i.encode('utf-8') for i in test_df['text']], dtype="object")
test_labels = test_df['labels'].to_numpy(dtype=int)

## Load pre-trained model (BERT)
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)

## Splitting datasets
x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

## Create new nn layers
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

#%%
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1,
                    callbacks=[callback])
model.save(os.getenv('AIP_MODEL_DIR'))

Writing training/train.py


In [9]:
%%writefile training/requirements.txt
tensorflow==2.11.0
tensorflow_hub
tensorflow-datasets
numpy
pandas
google-cloud-bigquery
db-dtypes

Writing training/requirements.txt


In [10]:
%%writefile training/Dockerfile
FROM nvidia/cuda:11.8.0-cudnn8-devel-ubuntu22.04
ARG DEBIAN_FRONTEND=noninteractive

COPY train.py train.py
COPY requirements.txt requirements.txt
RUN apt update -y
RUN apt-get install -y python3.10 && \
     apt-get install -y python3-pip
RUN pip install -r requirements.txt

CMD ["python3", "train.py"]

Writing training/Dockerfile


In [11]:
!docker build -t $IMAGE_URI training/.
!docker push $IMAGE_URI

Sending build context to Docker daemon  5.632kB
Step 1/8 : FROM nvidia/cuda:11.8.0-cudnn8-devel-ubuntu22.04
 ---> fd00873d2a37
Step 2/8 : ARG DEBIAN_FRONTEND=noninteractive
 ---> Using cache
 ---> 9e5c0c7ff153
Step 3/8 : COPY train.py train.py
 ---> d4799fbe429c
Step 4/8 : COPY requirements.txt requirements.txt
 ---> 6876f3ccf27d
Step 5/8 : RUN apt update -y
 ---> Running in 16d31eabde52


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [320 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [908 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [23.2 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [914 kB]
Get

In [19]:
aip.init(project=PROJECT_ID, staging_bucket=STAGING_FOLDER_URI)

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type" : "n1-standard-8",
            "accelerator_type": "NVIDIA_TESLA_T4",
            "accelerator_count": 1
        },
        "replica_count": "1",
        "container_spec": {
            "image_uri" : IMAGE_URI
        }
    }
]


job = aip.CustomJob(
    display_name="tensorflow-gpu-nlp",
    worker_pool_specs=worker_pool_specs,
    base_output_dir=MODEL_URI,
)

job.run()

Creating CustomJob
CustomJob created. Resource name: projects/254356041555/locations/us-central1/customJobs/5842437965434847232
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/254356041555/locations/us-central1/customJobs/5842437965434847232')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5842437965434847232?project=254356041555
CustomJob projects/254356041555/locations/us-central1/customJobs/5842437965434847232 current state:
2
CustomJob projects/254356041555/locations/us-central1/customJobs/5842437965434847232 current state:
2
CustomJob projects/254356041555/locations/us-central1/customJobs/5842437965434847232 current state:
2
CustomJob projects/254356041555/locations/us-central1/customJobs/5842437965434847232 current state:
2
CustomJob projects/254356041555/locations/us-central1/customJobs/5842437965434847232 current state:
2
CustomJob projects/254356041555/locations/us-central1/customJobs/58424

## Upload Model From Google Cloud Storage

### Create Container for Vertex Predictions

In [ ]:
!rm -fr prediction
!mkdir prediction

In [ ]:
!writefile

In [20]:
model = aip.Model.upload(
    display_name = 'cb-nlp-tf2',
    serving_container_image_uri = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-11:latest",
    artifact_uri = f'{MODEL_URI}/model',
)

Creating Model
Create Model backing LRO: projects/254356041555/locations/us-central1/models/8938671093028225024/operations/8272465796484038656
Model created. Resource name: projects/254356041555/locations/us-central1/models/8938671093028225024@1
To use this Model in another session:
model = aiplatform.Model('projects/254356041555/locations/us-central1/models/8938671093028225024@1')


In [22]:
endpoint = model.deploy(
    deployed_model_display_name = 'cb-nlp-tf2-end',
    traffic_percentage = 100,
    machine_type = 'n1-standard-4',
    min_replica_count = 1,
    max_replica_count = 1,
)

Creating Endpoint
Create Endpoint backing LRO: projects/254356041555/locations/us-central1/endpoints/5500632373391261696/operations/4629053697941307392
Endpoint created. Resource name: projects/254356041555/locations/us-central1/endpoints/5500632373391261696
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/254356041555/locations/us-central1/endpoints/5500632373391261696')
Deploying model to Endpoint : projects/254356041555/locations/us-central1/endpoints/5500632373391261696
Deploy Endpoint model backing LRO: projects/254356041555/locations/us-central1/endpoints/5500632373391261696/operations/7565400654986870784
Endpoint model deployed. Resource name: projects/254356041555/locations/us-central1/endpoints/5500632373391261696


In [27]:
## Testing

import numpy as np

string="This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
request = {"instances": np.array([string.encode('utf-8')], dtype='object')}
endpoint.predict(request)

Prediction(predictions=[[-0.768929064]], deployed_model_id='6501018629876744192', model_version_id='1', model_resource_name='projects/254356041555/locations/us-central1/models/8938671093028225024', explanations=None)

In [28]:
request

{'instances': array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."],
       dtype=object)}

![](./images/prediction.png)